# Python入门系列7——数据质量检查 

####  1. 重复值检查


重复值的检查首先要明确一点，即重复值的定义。对于一份二维表形式的数据集来说，什么是重复值？主要有两个层次：

① 关键字段出现重复记录，比如主索引字段出现重复；
② 所有字段出现重复记录。

第一个层次是否是重复，必须从这份数据的业务含义进行确定。比如一张表，从业务上讲，一个用户应该只会有一条记录，那么如果某个用户出现了超过一条的记录，那么这就是重复值。第二个层次就是最最最明显的重复值了。

重复值的产生主要有两个原因，一是上游源数据造成的，二是数据准备脚本中的数据关联造成的。从数据准备角度来看，首先检查数据准备的脚本，判断使用的源表是否有重复记录，同时检查关联语句的正确性和严谨性，比如关联条件是否合理、是否有限定数据周期等等。

如果是上游源数据出现重复，那么应该及时反映给上游进行修正；如果是脚本关联造成的，修改脚本，重新生成数据即可。

还有一份情况，这份数据集是一份单独的数据集，并不是在数据仓库中开发得到的数据，既没有上游源数据，也不存在生成数据的脚本，比如公开数据集，那么如何处理其中的重复值？一般的处理方式就是直接删除重复值。


In [ ]:
import pandas as pd 

dataset = pd.read_excel("数据路径")

#判断重复数据

dataset.duplicated()    #全部字段是否重复

dataset.duplicated(['主键'])    #主键是否重复




#删除重复数据

dataset.drop_duplicates()     #全部字段重复去重

dataset.drop_duplicates(['主键'])   #主键重复去重


#####  2. 缺失值检查


缺失值主要是指数据集中部分记录存在部分字段的信息缺失。




2.1 缺失值出现的三种原因

    ① 上游源系统因为技术或者成本原因无法完全获取到这一信息，比如对用户手机APP上网记录的解析；
    ② 从业务上讲，这一信息本来就不存在，比如一个学生的收入，一个未婚者的配偶姓名；
    ③ 数据准备脚本开发中的错误造成的。
    第一种原因，短期内无法解决；第二种原因，数据的缺失并不是错误，无法避免；第三种原因，则只需通过查证修改脚本即可。缺失值的存在既代表了某一部分信息的丢失，也影响了挖掘分析结论的可靠性与稳定性，因此，必须对缺失值进行处理。



In [ ]:
#查看哪些字段有缺失值   

dataset.isnull().any() 


#统计各字段的缺失值个数

missing_stat=dataset.isnull().apply(pd.value_counts)


#### 3. 异常值检查


异常值是指数据中出现了处于特定分布、范围或者趋势之外的数据，这些数据一般会被成为异常值、离群点、噪音等。




3.1 异常值产生的原因

    异常值的产生主要有两类原因：

① 数据采集、生成或者传递过程中发生的错误；

② 业务运营过程出现的一些特殊情况。
    将第一种原因产生的异常值称为统计上的异常，这是错误带来的数据问题，需要解决；将第二种原因产生的异常值称为业务上的异常，反映了业务运营过程的某种特殊结果，它不是错误，但需要深究，在数据挖掘中的一种典型应用就是异常检测模型，比如信用卡欺诈，网络入侵检测、客户异动行为识别等等。




3.2 异常值的识别方法

    异常值的识别方法主要有以下几种：




（1）极值检查

    主要检查字段的取值是否超出了合理的值域范围。


① 方法一：最大值最小值
    使用最大值、最小值进行判断。比如客户年龄的最大值为199岁，客户账单的最小费用为-20，这些都明显存在异常。


In [ ]:
statDF = dataset2.describe()  #获取描述性统计量

#获取各字段最大值、最小值

statDF.loc[['max','min']]

② 方法二：3σ原则
    如果数据服从正态分布，在3σ原则下，异常值被定义为与平均值的偏差超过了3倍标准差的值。这是因为，在正态分布的假设下，具体平均值3倍标准差之外的值出现的概率低于0.003，属于极个别的小概率事件。

In [ ]:
#计算相关统计指标

statDF = dataset2.describe()  #获取描述性统计量

#计算平均值+3倍标准差

statDF.loc['mean+3std']

= statDF.loc['mean'] + 3 * statDF.loc['std']  

#计算平均值-3倍标准差

statDF.loc['mean-3std'] 

= statDF.loc['mean'] - 3 * statDF.loc['std']  

③ 方法三：箱线图分析
    箱线图提供了识别异常的标准：异常值被定义为小于下四分位-1.5倍的四分位间距，或者大于上四分位+1.5倍的四分位间距的值。
箱线图分析不要求数据服从任何分布，因此对异常值的识别比较客观。

In [ ]:
#导入图像库

import matplotlib.pyplot as plt 

plt.rcParams['font.sans-serif'] = ['SimHei'] #用来正常显示中文标签

plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号

plt.figure() #建立图像

p = dataset.boxplot() #画箱线图，直接使用DataFrame的方法

（2）记录数分布检查

    主要检查字段的记录数分布是否超出合理的分布范围，包括三个指标：零值记录数、正值记录数、负值记录数。




（3）波动检查

    波动检查主要适用于有监督的数据，用于检查随着自变量的变化，因变量是否发生明显的波动情况。
    以上异常值的识别方法主要针对连续型的字段，而对于离散型的字段的异常识别主要通过检查类别出现是否出现了合理阈值外的数据，比如苹果终端型号字段，出现了“P20”的取值。




3.3 异常值的处理

    对于统计上的异常值的处理，主要采取两种方式：剔除或者替换。剔除是指直接将被标记为异常值的记录从数据集中删除掉，而替换是指将异常值用一个非异常值进行替换，比如边界值，或者有监督情况下的目标变量表征相似的某个值。
    对于业务上的异常值的处理，原则就是进行深入探索分析，查找出现这一特殊情况的根本原因。
